## Deliverable 2. Create a Customer Travel Destinations Map.

In [83]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [16]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Alamogordo,US,2022-08-08 15:52:35,32.8995,-105.9603,84.70,48,0,6.91,clear sky
1,1,Rikitea,PF,2022-08-08 15:52:35,-23.1203,-134.9692,72.25,80,100,28.21,moderate rain
2,2,Jumla,NP,2022-08-08 15:52:35,29.2747,82.1838,64.53,89,89,3.31,overcast clouds
3,3,Upernavik,GL,2022-08-08 15:52:36,72.7868,-56.1549,42.76,75,99,4.61,overcast clouds
4,4,Sao Filipe,CV,2022-08-08 15:52:36,14.8961,-24.4956,79.86,83,94,7.02,light rain


In [17]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


In [18]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Alamogordo,US,2022-08-08 15:52:35,32.8995,-105.9603,84.70,48,0,6.91,clear sky
1,1,Rikitea,PF,2022-08-08 15:52:35,-23.1203,-134.9692,72.25,80,100,28.21,moderate rain
4,4,Sao Filipe,CV,2022-08-08 15:52:36,14.8961,-24.4956,79.86,83,94,7.02,light rain
15,15,West Sacramento,US,2022-08-08 15:50:10,38.5805,-121.5302,73.49,72,0,10.36,clear sky
16,16,Puerto Ayora,EC,2022-08-08 15:52:40,-0.7393,-90.3518,71.55,83,90,8.99,overcast clouds
22,22,Butaritari,KI,2022-08-08 15:52:43,3.0707,172.7902,80.55,78,22,12.50,light rain
23,23,Umm Lajj,SA,2022-08-08 15:52:43,25.0213,37.2685,89.91,62,22,10.87,few clouds
24,24,Georgetown,MY,2022-08-08 15:49:32,5.4112,100.3354,84.88,74,20,3.44,light rain
25,25,Ponta Do Sol,PT,2022-08-08 15:52:43,32.6667,-17.1000,81.72,66,2,4.21,clear sky
27,27,Ambulu,ID,2022-08-08 15:52:46,-8.3450,113.6058,75.15,88,98,4.70,overcast clouds


In [19]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                270
City                   270
Country                266
Date                   270
Lat                    270
Lng                    270
Max Temp               270
Humidity               270
Cloudiness             270
Wind Speed             270
Current Description    270
dtype: int64

In [20]:
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                4
Date                   0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [30]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [37]:
clean_df.isnull().sum()

City_ID                0
City                   0
Country                0
Date                   0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [38]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Alamogordo,US,84.70,clear sky,32.8995,-105.9603,
1,Rikitea,PF,72.25,moderate rain,-23.1203,-134.9692,
4,Sao Filipe,CV,79.86,light rain,14.8961,-24.4956,
15,West Sacramento,US,73.49,clear sky,38.5805,-121.5302,
16,Puerto Ayora,EC,71.55,overcast clouds,-0.7393,-90.3518,
22,Butaritari,KI,80.55,light rain,3.0707,172.7902,
23,Umm Lajj,SA,89.91,few clouds,25.0213,37.2685,
24,Georgetown,MY,84.88,light rain,5.4112,100.3354,
25,Ponta Do Sol,PT,81.72,clear sky,32.6667,-17.1000,
27,Ambulu,ID,75.15,overcast clouds,-8.3450,113.6058,


In [39]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
   

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [97]:
# 7. Drop the rows where there is no Hotel Name.
hotel_nan = hotel_df.replace('', np.nan)
clean_hotel_df = hotel_nan.dropna()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Alamogordo,US,84.70,clear sky,32.8995,-105.9603,Fairfield Inn & Suites by Marriott Alamogordo
1,Rikitea,PF,72.25,moderate rain,-23.1203,-134.9692,People ThankYou
4,Sao Filipe,CV,79.86,light rain,14.8961,-24.4956,Hotel Xaguate
15,West Sacramento,US,73.49,clear sky,38.5805,-121.5302,"Holiday Inn Sacramento Downtown - Arena, an IH..."
16,Puerto Ayora,EC,71.55,overcast clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
...,...,...,...,...,...,...,...
546,Kathmandu,NP,73.62,broken clouds,27.7167,85.3167,Hotel Shanker Kathmandu
549,Tautira,PF,74.50,overcast clouds,-17.7333,-149.1500,Pension Oaoa
550,Gay,RU,70.99,overcast clouds,51.4666,58.4552,Hotel Berezka
555,Teguise,ES,80.82,few clouds,29.0605,-13.5640,B&B La Casona de Nazaret


In [98]:
clean_hotel_df.isnull().sum()

City                   0
Country                0
Max Temp               0
Current Description    0
Lat                    0
Lng                    0
Hotel Name             0
dtype: int64

In [99]:
clean_hotel_df.count()

City                   248
Country                248
Max Temp               248
Current Description    248
Lat                    248
Lng                    248
Hotel Name             248
dtype: int64

In [94]:
clean_hotel_df.head(16)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Alamogordo,US,84.70,clear sky,32.8995,-105.9603,Fairfield Inn & Suites by Marriott Alamogordo
1,Rikitea,PF,72.25,moderate rain,-23.1203,-134.9692,People ThankYou
4,Sao Filipe,CV,79.86,light rain,14.8961,-24.4956,Hotel Xaguate
15,West Sacramento,US,73.49,clear sky,38.5805,-121.5302,"Holiday Inn Sacramento Downtown - Arena, an IH..."
16,Puerto Ayora,EC,71.55,overcast clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
22,Butaritari,KI,80.55,light rain,3.0707,172.7902,Isles Sunset Lodge
23,Umm Lajj,SA,89.91,few clouds,25.0213,37.2685,Moon light Furnished Units
24,Georgetown,MY,84.88,light rain,5.4112,100.3354,Cititel Penang
25,Ponta Do Sol,PT,81.72,clear sky,32.6667,-17.1000,Hotel do Campo
27,Ambulu,ID,75.15,overcast clouds,-8.3450,113.6058,Hotel Ambulu Jember


In [104]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [100]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [101]:
# 11a. Add a marker layer for each city to the map. 

locations = clean_hotel_df[["Lat", "Lng"]]
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))